In [93]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [94]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [95]:
df = pd.read_csv("ratings.csv")

In [96]:
df = df.drop(["userId"], axis = 1)
df = df.loc[:, (df != 0).any(axis=0)]
X_train, X_test = train_test_split(df, train_size=0.8)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [100]:
n_nodes_inpl = 5379
n_nodes_hl1  = 256  
n_nodes_outl = 5379  
# first hidden layer has 784*32 weights and 32 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
# first hidden layer has 784*32 weights and 32 biases
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl])) }

In [101]:
# user with 3706 ratings goes in
input_layer = tf.placeholder('float', [None, 5379])
# add a constant node to the first layer
# it needs to have the same shape as the input layer for me to be
# able to concatinate it later
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,hidden_1_layer_vals['weights']))
# adding one bias node to the hidden layer
layer1_const = tf.fill([tf.shape(layer_1)[0], 1] ,1.0 )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, 5379])
# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define our optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdamOptimizer().minimize(meansq)#AdagradOptimizer(learn_rate).minimize(meansq)

In [102]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# defining batch size, number of epochs and learning rate
batch_size = 100  # how many images to use together for training
hm_epochs = 2000   # how many times to go through the entire dataset
tot_users = X_train.shape[0] # total number of images

In [103]:
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    
    for i in range(int(tot_users/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq], feed_dict={input_layer: epoch_x, output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer,feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer,feed_dict={input_layer:X_test})
        
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

MSE train 115.98475048091439 MSE test 116.23830184233651
Epoch 0 / 2000 loss: 486.37506103515625
MSE train 108.28471108307666 MSE test 108.84362155910819
Epoch 1 / 2000 loss: 451.8434524536133
MSE train 101.51888612686021 MSE test 102.33527594209279
Epoch 2 / 2000 loss: 421.9932098388672
MSE train 95.58042119350574 MSE test 96.65087655934526
Epoch 3 / 2000 loss: 395.88072204589844
MSE train 90.42173733511791 MSE test 91.70333053983153
Epoch 4 / 2000 loss: 373.0421600341797
MSE train 85.91246216508408 MSE test 87.34081479496771
Epoch 5 / 2000 loss: 353.2476348876953
MSE train 81.93850245862245 MSE test 83.49280340304227
Epoch 6 / 2000 loss: 335.9014587402344
MSE train 78.43930349489489 MSE test 80.08020197734629
Epoch 7 / 2000 loss: 320.5614547729492
MSE train 75.34444974529013 MSE test 77.08902122884491
Epoch 8 / 2000 loss: 307.15931701660156
MSE train 72.61350529635723 MSE test 74.4529179351216
Epoch 9 / 2000 loss: 295.19200897216797
MSE train 70.19192527923657 MSE test 72.14069982866

In [91]:
saver = tf.train.Saver()
#Run the operation by feeding input
saver.save(sess, 'my_test_model.h5',global_step=1000)

'my_test_model.h5-1000'

In [86]:
# pick a user
sample_user = X_test.iloc[90,:]
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})

In [87]:
movies = pd.read_csv("ml-latest-small/movies.csv")
movies_Id = movies.movieId
movies_title = movies.title
id2title = dict(zip(movies_Id, movies_title))

In [88]:
sample_user_pred

array([[ 6.8548098 , -0.47716236,  0.13501167, ...,  1.1837462 ,
        -0.23169938, -2.1738634 ]], dtype=float32)

In [89]:
movie_rankings = sorted(dict(zip(df.columns.values, sample_user_pred[0])))
movie_rankings = [int(i) for i in movie_rankings]
movie_rankings_with_name = []
for i in movie_rankings:
    if i in id2title.keys():
        print(id2title[i])
        movie_rankings_with_name = [id2title[i] ]

Toy Story (1995)
GoldenEye (1995)
City Hall (1996)
Extreme Measures (1996)
Glimmer Man, The (1996)
D3: The Mighty Ducks (1996)
Chamber, The (1996)
Apple Dumpling Gang, The (1975)
Davy Crockett, King of the Wild Frontier (1955)
Escape to Witch Mountain (1975)
Bottle Rocket (1996)
Love Bug, The (1969)
Herbie Rides Again (1974)
Old Yeller (1957)
Parent Trap, The (1961)
Pollyanna (1960)
Homeward Bound: The Incredible Journey (1993)
Shaggy Dog, The (1959)
Swiss Family Robinson (1960)
That Darn Cat! (1965)
20,000 Leagues Under the Sea (1954)
Mr. Wrong (1996)
Cool Runnings (1993)
Angels in the Outfield (1994)
Cinderella (1950)
Winnie the Pooh and the Blustery Day (1968)
Three Caballeros, The (1945)
Sword in the Stone, The (1963)
Robin Hood: Prince of Thieves (1991)
Mary Poppins (1964)
Dumbo (1941)
Unforgettable (1996)
Pete's Dragon (1977)
Bedknobs and Broomsticks (1971)
Alice in Wonderland (1951)
Fox and the Hound, The (1981)
Freeway (1996)
Sound of Music, The (1965)
Die Hard (1988)
Lawnmower

In [ ]:
tf.keras